In [25]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm

In [27]:
df = pd.read_stata('Toutlamonde2022.dta') # Display the first few rows of the DataFrame 
print(df.head())

       hhid  Tot_exp  Tot_exp_online   cons_pan  cons_bidi  cons_cigarettes  \
0  31000301  13000.0           500.0   0.000000        0.0         0.000000   
1  31000302  39000.0          3500.0   7.744000        0.0        53.240002   
2  31000306  18000.0          1000.0  19.360001        0.0        58.080002   
3  31000309  29000.0           800.0   0.000000        0.0        58.080002   
4  31001302  19500.0             0.0  36.299999        0.0         0.000000   

   cons_leaf_tobacco  cons_toddy  cons_country_liquor   cons_beer  ...  \
0                0.0         0.0                  0.0  387.200012  ...   
1                0.0         0.0                  0.0  425.920013  ...   
2                0.0         0.0                  0.0  193.600006  ...   
3                0.0         0.0                  0.0    0.000000  ...   
4                0.0         0.0                  0.0    0.000000  ...   

   total_exp_2022_under  rural  pred_cons_exp_WBmodel  \
0                6292.0

In [29]:
categorical_vars = ['marital_status', 'edu_level', 'hoh_religion', 'hh_type', 'caste', 'land_type', 'type_house', 'material_wall','material_roof','material_floor','source_cooking','source_lighting','source_water','type_latrine','type_rationcard']

# Convert to dummy variables
data = pd.get_dummies(df, columns=categorical_vars, drop_first=True)

# Check the first few rows to confirm dummy variables were created
print(data.columns.tolist())

['hhid', 'Tot_exp', 'Tot_exp_online', 'cons_pan', 'cons_bidi', 'cons_cigarettes', 'cons_leaf_tobacco', 'cons_toddy', 'cons_country_liquor', 'cons_beer', 'cons_refined_liquor', 'multiplier', 'cons_tobacco_tot', 'cons_alcohol_tot', 'cons_clothes_tot', 'cons_misc_services_tot', 'year', 'sector', 'state', 'district', 'age', 'edu_years', 'used_internet', 'days_outside_home', 'meals_school', 'hh_size', 'employed_annual', 'level_access_latrine', 'cons_facemask_gloves', 'cons_western_suit', 'cons_school_uniform_boys', 'cons_school_uniform_girls', 'cons_clothing_others', 'cons_clothing_secondhand', 'cons_pillow_cushion', 'cons_quilt_mattress', 'cons_mosquito_net', 'cons_bedding_tot', 'cons_leather_shoes', 'cons_leather_chappals', 'cons_rubber_footwear', 'cons_footwear_secondhand', 'cons_footwear_tot', 'online_clothing', 'online_footwear', 'online_personal_goods', 'online_rec', 'online_cooking', 'online_medical', 'has_tv', 'has_radio', 'has_laptop', 'has_mobile', 'has_bicycle', 'has_bike', 'has_

In [31]:
data = data.replace({True: 1, False: 0})

# Confirm the replacement
print(data.head())


/var/folders/3w/5w6s9nld1y39p5mrz9f82ywm0000gn/T/ipykernel_52374/2654137366.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace({True: 1, False: 0})


       hhid  Tot_exp  Tot_exp_online   cons_pan  cons_bidi  cons_cigarettes  \
0  31000301  13000.0           500.0   0.000000        0.0         0.000000   
1  31000302  39000.0          3500.0   7.744000        0.0        53.240002   
2  31000306  18000.0          1000.0  19.360001        0.0        58.080002   
3  31000309  29000.0           800.0   0.000000        0.0        58.080002   
4  31001302  19500.0             0.0  36.299999        0.0         0.000000   

   cons_leaf_tobacco  cons_toddy  cons_country_liquor   cons_beer  ...  \
0                0.0         0.0                  0.0  387.200012  ...   
1                0.0         0.0                  0.0  425.920013  ...   
2                0.0         0.0                  0.0  193.600006  ...   
3                0.0         0.0                  0.0    0.000000  ...   
4                0.0         0.0                  0.0    0.000000  ...   

   type_latrine_5  type_latrine_6  type_latrine_7  type_latrine_8  \
0          

In [32]:
print("Checking for missing values (NaN):")
print(data.isnull().sum())  # Count missing values per column

print("\nChecking for infinite values:")
print((data == float('inf')).sum())  # Count positive infinity values
print((data == -float('inf')).sum())  # Count negative infinity values


Checking for missing values (NaN):
hhid                 0
Tot_exp              0
Tot_exp_online       0
cons_pan             0
cons_bidi            0
                    ..
type_rationcard_2    0
type_rationcard_3    0
type_rationcard_4    0
type_rationcard_5    0
type_rationcard_9    0
Length: 293, dtype: int64

Checking for infinite values:
hhid                 0
Tot_exp              0
Tot_exp_online       0
cons_pan             0
cons_bidi            0
                    ..
type_rationcard_2    0
type_rationcard_3    0
type_rationcard_4    0
type_rationcard_5    0
type_rationcard_9    0
Length: 293, dtype: int64
hhid                 0
Tot_exp              0
Tot_exp_online       0
cons_pan             0
cons_bidi            0
                    ..
type_rationcard_2    0
type_rationcard_3    0
type_rationcard_4    0
type_rationcard_5    0
type_rationcard_9    0
Length: 293, dtype: int64


In [35]:
data.replace([float('inf'), -float('inf')], pd.NA, inplace=True)



In [37]:
dependent_var = "total_exp_2022"
# Update independent variables

independent_vars = [
    "cons_pan", "cons_bidi", "cons_cigarettes", "cons_leaf_tobacco", "cons_toddy", "cons_country_liquor", 
    "cons_beer", "cons_refined_liquor", "cons_tobacco_tot", "cons_alcohol_tot", "cons_clothes_tot", 
    "cons_misc_services_tot", "year", "sector", "age", "used_internet", 
    "hh_size", "employed_annual", "cons_facemask_gloves", 
    "cons_western_suit", "cons_school_uniform_boys", "cons_school_uniform_girls", "cons_clothing_others", 
    "cons_clothing_secondhand", "cons_pillow_cushion", "cons_quilt_mattress", "cons_mosquito_net", 
    "cons_bedding_tot", "cons_leather_shoes", "cons_leather_chappals", "cons_rubber_footwear", 
    "cons_footwear_secondhand", "cons_footwear_tot", "online_clothing", "online_footwear", 
    "online_personal_goods", "online_rec", "online_cooking", "online_medical", "has_tv", "has_radio", 
    "has_laptop", "has_mobile", "has_bicycle", "has_bike", "has_car", "has_truck", "has_animalcart", 
    "has_fridge", "has_washingmachine", "has_ac", "cons_LED_bulb", "cons_tuition_fees", "cons_education_tot", 
    "cons_medicine", "cons_Xray_ECG_tests", "cons_doctor_fees", "cons_medical_insti_tot", 
    "cons_medical_non_insti_tot", "cons_cinema", "cons_cable_TV", "cons_entertainment_tot", "cons_pads", 
    "cons_other_electric_light", "cons_other_HH_consumables_tot", "cons_landline", "cons_mobile", 
    "cons_internet", "cons_water_bill", "cons_other_fuel", "cons_firewood", "cons_electricity", 
    "cons_kerosene_PDS", "cons_kerosene_other", "cons_LPG", "cons_fuel_light_tot", "procured_kerosene_ration", 
    "receieved_subsidy_lpg", "num_subsidized_lpg", "num_free_oth", "is_hospitalization", "online_fuel", 
    "online_edu", "has_internet", "cons_bakery", "cons_health_supplements", "cons_cooked_snacks_purchased", 
    "cons_S_processed_food_tot", "cons_P_processed_food_tot", "used_ration", "online_groceries", 
    "online_veg", "online_fruits", "online_eggfishmeat", "online_processed_packed", "cons_coarse_grains_PDS", 
    "cons_coarse_grains_other", "cons_rice_free", "cons_atta_free", "cons_coarse_grains_free", 
    "cons_pulses_free", "cons_salt_free", "cons_sugar_free", "cons_vanaspati_oil", "cons_rice_other", 
    "cons_rice_PDS", "cons_attaPDS", "cons_attaNormal", "cons_maida", "cons_cereal_tot", "cons_arhar_tur", 
    "cons_moong", "cons_masur", "cons_urd", "cons_pulses_PDS", "cons_pulses_tot", "cons_milk", 
    "cons_condensed_milk", "cons_ghee", "cons_milk_tot", "cons_sugarPDS", "cons_gur", "cons_honey", 
    "cons_salt_PDS", "cons_sugar_salt_tot", "cons_refined_oil", "cons_oil_PDS", "cons_oil_tot", 
    "cons_egg_meat_tot", "cons_potato", "cons_onion", "cons_tomato", "cons_brinjal", "cons_radish", 
    "cons_carrot", "cons_leafyveggie", "cons_green_chillies", "cons_ladyfinger", "cons_parwal", 
    "cons_cauliflower", "cons_cabbage", "cons_gourds_pumpkin", "cons_peas", "cons_beans", "cons_lemon", 
    "cons_otherveggie", "cons_fruits_tot", "cons_dryfruits_tot", "cons_spices_tot", "cons_mineral_water", 
    "hh_is_PMJAY", "recd_fee_waiver", "hh_pmgky", "male", "owns_land", "hh_has_house", "free_electricity", 
    "rural", "marital_status_2", "marital_status_3", "marital_status_4", "hoh_religion_1", "hoh_religion_2", "hoh_religion_3", 
    "hoh_religion_4", "hoh_religion_5", "hoh_religion_6", "hoh_religion_7", "hoh_religion_9", 
    "hh_type_2", "hh_type_3", "hh_type_4", "hh_type_5", "hh_type_6", "hh_type_9", "caste_1", "caste_2", 
    "caste_3", "caste_9", "land_type_1", "land_type_2", "land_type_3", "type_house_2", "type_house_3", 
    "material_wall_2", "material_wall_3", "material_wall_4", "material_wall_5", "material_wall_6", 
    "material_wall_7", "material_wall_8", "material_wall_9", "material_roof_2", "material_roof_3", 
    "material_roof_4", "material_roof_5", "material_roof_6", "material_roof_7", "material_roof_8", 
    "material_roof_9", "material_floor_2", "material_floor_3", "material_floor_4", "material_floor_5", 
    "material_floor_6", "material_floor_7", "material_floor_8", "material_floor_9", "source_cooking_10", 
    "source_cooking_11", "source_cooking_12", "source_cooking_2", "source_cooking_3", "source_cooking_4", 
    "source_cooking_5", "source_cooking_6", "source_cooking_7", "source_cooking_8", "source_cooking_9", 
    "source_lighting_2", "source_lighting_3", "source_lighting_4", "source_lighting_5", "source_lighting_9", 
    "source_water_10", "source_water_11", "source_water_12", "source_water_13", "source_water_14", 
    "source_water_15", "source_water_16", "source_water_19", "source_water_2", "source_water_3", 
    "source_water_4", "source_water_5", "source_water_6", "source_water_7", "source_water_8", 
    "source_water_9", "type_rationcard_1", "type_rationcard_2", "type_rationcard_3", 
    "type_rationcard_4", "type_rationcard_5", "type_rationcard_9"
]


# Initialize a dictionary to store adjusted R^2 values
adjusted_r2_results = {}

# Loop through each independent variable
for var in independent_vars:
    X = sm.add_constant(data[[var]])  # Add a constant for the intercept
    y = data[dependent_var]
    
    # Fit the model
    model = sm.OLS(y, X).fit()
    
    # Compute adjusted R^2
    adjusted_r2 = model.rsquared_adj
    adjusted_r2_results[var] = adjusted_r2

# Find the variable with the highest adjusted R^2
best_feature = max(adjusted_r2_results, key=adjusted_r2_results.get)
best_adj_r2 = adjusted_r2_results[best_feature]

# Print the result
print(f"The feature with the highest adjusted R^2 is: {best_feature}")
print(f"Adjusted R^2: {best_adj_r2}")

The feature with the highest adjusted R^2 is: cons_fuel_light_tot
Adjusted R^2: 0.2455707438229685


In [39]:
for var in independent_vars:
    if var not in data.columns:  # Skip missing variables
        print(f"Variable {var} is missing from the dataset.")
        continue
    
    X = sm.add_constant(data[[var]])
    y = data[dependent_var]
    
    if X.isnull().values.any() or not np.isfinite(X).all().all():
        print(f"Variable {var} contains NaN or infinite values.")
    else:
        print(f"Variable {var} is clean.")
        

Variable cons_pan is clean.
Variable cons_bidi is clean.
Variable cons_cigarettes is clean.
Variable cons_leaf_tobacco is clean.
Variable cons_toddy is clean.
Variable cons_country_liquor is clean.
Variable cons_beer is clean.
Variable cons_refined_liquor is clean.
Variable cons_tobacco_tot is clean.
Variable cons_alcohol_tot is clean.
Variable cons_clothes_tot is clean.
Variable cons_misc_services_tot is clean.
Variable year is clean.
Variable sector is clean.
Variable age is clean.
Variable used_internet is clean.
Variable hh_size is clean.
Variable employed_annual is clean.
Variable cons_facemask_gloves is clean.
Variable cons_western_suit is clean.
Variable cons_school_uniform_boys is clean.
Variable cons_school_uniform_girls is clean.
Variable cons_clothing_others is clean.
Variable cons_clothing_secondhand is clean.
Variable cons_pillow_cushion is clean.
Variable cons_quilt_mattress is clean.
Variable cons_mosquito_net is clean.
Variable cons_bedding_tot is clean.
Variable cons_l

In [41]:
missing_vars = [var for var in independent_vars if var not in data.columns]
if dependent_var not in data.columns:
    print(f"Dependent variable '{dependent_var}' is missing in the DataFrame.")
if missing_vars:
    print(f"The following independent variables are missing in the DataFrame: {missing_vars}")
else:
    print("All variables are present in the DataFrame.")


All variables are present in the DataFrame.


In [43]:
for var in independent_vars + [dependent_var]:
    if var in data.columns:
        if data[var].isnull().sum() > 0:
            print(f"Variable '{var}' has {data[var].isnull().sum()} missing values.")


In [45]:
non_numeric_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"Non-numeric columns: {non_numeric_cols}")


Non-numeric columns: ['level_access_latrine']


In [ ]:
fixed_vars = ["cons_fuel_light_tot"]
remaining_vars = [var for var in independent_vars if var not in fixed_vars]

# Store results for each step
stepwise_results = []

while remaining_vars:
    best_adj_r2 = -float("inf")
    best_var = None

    # Evaluate adjusted R^2 for adding each remaining variable
    for var in remaining_vars:
        X = sm.add_constant(data[fixed_vars + [var]])  # Add constant and current variables
        y = data[dependent_var]
        
        # Fit the model
        model = sm.OLS(y, X).fit()
        
        # Compute adjusted R^2
        adjusted_r2 = model.rsquared_adj
        
        # Keep track of the variable with the highest adjusted R^2
        if adjusted_r2 > best_adj_r2:
            best_adj_r2 = adjusted_r2
            best_var = var

    # Add the best variable to fixed variables
    if best_var:
        fixed_vars.append(best_var)
        remaining_vars.remove(best_var)
        stepwise_results.append((best_var, best_adj_r2))
    else:
        break

# Output the results
for step, (variable, adj_r2) in enumerate(stepwise_results, start=1):
    print(f"Step {step}: Added variable '{variable}' with Adjusted R^2 = {adj_r2}")

print("\nFinal selected variables:", fixed_vars)